Here I have made 4 dataframes which I have imported below and then run analysis for the same.

In [1]:
import pandas as pd
import time
from IPython.display import display
import numpy as np
import copy
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
import math
import matplotlib.pyplot as plt
import requests
import io

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [2]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

requests==2.24.0
plotly==4.14.1
pandas==1.0.5
numpy==1.18.5
matplotlib==3.2.2
cufflinks==0.17.3


### Data Importing

In [3]:
url_Nifty50_CAGR = "https://raw.githubusercontent.com/sarthakgoel31/Portfolio-outperformance-in-terms-of-CAGR/main/Nifty50%20CAGR.csv"
download_Nifty50_CAGR = requests.get(url_Nifty50_CAGR).content
Nifty50_CAGR = pd.read_csv(io.StringIO(download_Nifty50_CAGR.decode('utf-8')))

url_Portfolio_CAGR = "https://raw.githubusercontent.com/sarthakgoel31/Portfolio-outperformance-in-terms-of-CAGR/main/Portfolio%20CAGR.csv"
download_Portfolio_CAGR = requests.get(url_Portfolio_CAGR).content
Portfolio_CAGR = pd.read_csv(io.StringIO(download_Portfolio_CAGR.decode('utf-8')))

url_Sectoral_CAGR = "https://raw.githubusercontent.com/sarthakgoel31/Portfolio-outperformance-in-terms-of-CAGR/main/Sectoral%20CAGR.csv"
download_Sectoral_CAGR = requests.get(url_Sectoral_CAGR).content
Sectoral_CAGR = pd.read_csv(io.StringIO(download_Sectoral_CAGR.decode('utf-8')))

url_Stock_CAGR = "https://raw.githubusercontent.com/sarthakgoel31/Portfolio-outperformance-in-terms-of-CAGR/main/Stock%20CAGR.csv"
download_Stock_CAGR = requests.get(url_Stock_CAGR).content
Stock_CAGR = pd.read_csv(io.StringIO(download_Stock_CAGR.decode('utf-8')))

Stock_CAGR['Nifty50 Weightage'] = Stock_CAGR['Nifty50 Weightage'].str.replace('%', '')
Stock_CAGR['Nifty50 Weightage'] = Stock_CAGR['Nifty50 Weightage'].astype(float)

Stock_CAGR['Portfolio Weightage'] = Stock_CAGR['Portfolio Weightage'].str.replace('%', '')
Stock_CAGR['Portfolio Weightage'] = Stock_CAGR['Portfolio Weightage'].astype(float)

Sectoral_CAGR['Nifty50 Weightage'] = Sectoral_CAGR['Nifty50 Weightage'].str.replace('%', '')
Sectoral_CAGR['Nifty50 Weightage'] = Sectoral_CAGR['Nifty50 Weightage'].astype(float)

Sectoral_CAGR['Portfolio Weightage'] = Sectoral_CAGR['Portfolio Weightage'].str.replace('%', '')
Sectoral_CAGR['Portfolio Weightage'] = Sectoral_CAGR['Portfolio Weightage'].astype(float)

Nifty50_CAGR represents all the stocks in Nifty50, their sectors, their weightage in Nifty50, their CAGR in the past year and Weighted Return (CAGR * Weight).
I calculated the CAGR by downoading the Historical Data from yahoo Finance and then computing the same.

In [4]:
Nifty50_CAGR

,Name,Sectoral,CAGR,Weightage,Weighted Return
0,Adani Ports & Special Economic Zone,Transportation,1.028,0.63%,0.006476
1,Asian Paints Ltd,Consumer Non Durables,0.318,1.84%,0.005851
2,Axis Bank Ltd,Banks,0.121,2.79%,0.003376
3,Bajaj Auto Ltd,Auto,0.465,0.89%,0.004138
4,Bajaj Finance Ltd,Finance,0.241,2.13%,0.005133
5,Bajaj Finserv Ltd,Finance,0.145,0.90%,0.001305
6,Bharat Petroleum Corporation Ltd,Petroleum Products,0.086,0.52%,0.000447
7,Bharti Airtel Ltd,Telecom - Services,0.112,2.26%,0.002531
8,Britannia Industries Ltd,Consumer Non Durables,0.125,0.70%,0.000875
9,Cipla Ltd,Pharmaceuticals,0.900,0.71%,0.006390


Portfolio_CAGR represents all the stocks in the Canara Robeco Blue Chip Equity fund (Growth), their sectors, their weightage in Nifty50, their CAGR in the past year and Weighted Return (CAGR * Weight).
I calculated the CAGR by downoading the Historical Data from yahoo Finance and then computing the same.

In [5]:
Portfolio_CAGR

,Name,Sector,CAGR,Weightage,Weighted Return
0,Abbott India Ltd,Pharmaceuticals,-0.070,0.76%,-0.000532
1,Asian Paints Ltd,Consumer Non Durables,0.318,0.83%,0.002639
2,Avenue Supermarts Ltd,Retailing,0.383,1.26%,0.004826
3,Axis Bank Ltd,Banks,0.121,3.36%,0.004066
4,Bajaj Auto Ltd,Auto,0.465,1.76%,0.008184
5,Bajaj Finance Ltd,Finance,0.241,3.72%,0.008965
6,Bharti Airtel Ltd,Telecom - Services,0.112,3.06%,0.003427
7,Britannia Industries Ltd,Consumer Non Durables,0.125,1.21%,0.001512
8,Cipla Ltd,Pharmaceuticals,0.900,1.33%,0.011970
9,Crompton Greaves Consumer Electricals Ltd,Consumer Durables,0.380,1.76%,0.006688


Stock_CAGR represents all the stocks in our universe of the project, their weightage in Nifty50 and Portfolio, their CAGR in the past year and Weighted Return (CAGR * Weight) for both Nifty50 and Portfolio.
I calculated the CAGR by downoading the Historical Data from yahoo Finance and then computing the same.

In [6]:
Stock_CAGR

,Stocks,CAGR,Nifty50 Weightage,Nifty50 CAGR,Portfolio Weightage,Portfolio CAGR
0,Asian Paints Ltd,0.318,1.84,0.005851,0.83,0.002639
1,Abbott India Ltd,-0.070,0.00,0.000000,0.76,-0.000532
2,Adani Ports & Special Economic Zone,1.028,0.63,0.006476,0.00,0.000000
3,Avenue Supermarts Ltd,0.383,0.00,0.000000,1.26,0.004826
4,Axis Bank Ltd,0.121,2.79,0.003376,3.36,0.004066
5,Bajaj Auto Ltd,0.465,0.89,0.004138,1.76,0.008184
6,Bajaj Finance Ltd,0.241,2.13,0.005133,3.72,0.008965
7,Bajaj Finserv Ltd,0.145,0.90,0.001305,0.00,0.000000
8,Bharat Petroleum Corporation Ltd,0.086,0.52,0.000447,0.00,0.000000
9,Bharti Airtel Ltd,0.112,2.26,0.002531,3.06,0.003427


Sectoral_CAGR represents all the sectors in our universe of the project, their weightage in Nifty50 and Portfolio, and their CAGR in the past year for both Nifty50 and Portfolio which is summation of all the CAGR of stocks in that particular sector for that portfolio/benchmark.
I calculated the CAGR by downoading the Historical Data from yahoo Finance and then computing the same.

In [7]:
Sectoral_CAGR

,Sectors,Nifty50 Weightage,Nifty50 CAGR,Portfolio Weightage,Portfolio CAGR
0,Banks,26.27,0.068572,23.67,0.064510
1,Software,17.09,0.104073,15.63,0.099994
2,Construction Project,2.74,0.007535,3.37,0.009267
3,Petroleum Products,11.03,0.062108,5.53,0.033844
4,Finance,11.77,0.025341,10.50,0.027872
5,Telecom - Services,2.26,0.002531,3.06,0.003427
6,Auto,5.91,0.035837,6.05,0.025002
7,Pharmaceuticals,3.46,0.020935,5.92,0.031738
8,Consumer Non Durables,10.02,0.008923,8.35,0.016988
9,Cement,2.26,0.011579,2.91,0.011228


### Analysis...

You can zoom in the charts to checkout different stocks/sectors deeply.

Stock-Wise Weightage Bar Chart

In [8]:
df = Stock_CAGR[[('Nifty50 Weightage'), ('Portfolio Weightage')]].set_index(Stock_CAGR['Stocks'])

df.iplot(kind='bar', xTitle='Stocks', yTitle='Weightage', title='Stock-Wise Weightage')

Stock-Wise CAGR Performance Bar Chart

In [9]:
df = Stock_CAGR[['Nifty50 CAGR','Portfolio CAGR']].set_index(Stock_CAGR['Stocks'])

df.iplot(kind='bar', xTitle='Stocks', yTitle='CAGR', title='Stock-Wise CAGR performances')

Sectoral-Wise Weightage Bar Chart

In [10]:
df = Sectoral_CAGR[['Nifty50 Weightage', 'Portfolio Weightage']].set_index(Sectoral_CAGR['Sectors'])

df.iplot(kind='bar', xTitle='Sectors', yTitle='Weightage', title='Sector-Wise Weightage')

Sectoral-Wise CAGR Performance Bar Chart

In [11]:
df = Sectoral_CAGR[['Nifty50 CAGR', 'Portfolio CAGR']].set_index(Sectoral_CAGR['Sectors'])

df.iplot(kind='bar', xTitle='Sectors', yTitle='CAGR', title='Sector-Wise CAGR performances')

### Our Conclusion 

We see how Nifty50 has performed better than the Portfolio, even though we understand that there was a pandemic and all of the stocks crashed, but as per our analysis mostly all of them have either come back to where they were an year ago or have gone in green.

Seeing, this we see that the portfolio is expected to perform better than the benchmark and here we have seen that both have performed equally well, or some might say that Benchmark surpassed the Portfolio.

As an investor, I might not look in this portfolio as the performance has not been of the calibre that you expect from a Fund.